** Self-Assembly of a Dimer System -- Companion Notebook -- December 2018 **

*** Biophysical Parameter Calculations ***

In [1]:
%matplotlib inline
import numpy as np
import scipy 
import matplotlib.pyplot as plt
from scipy.optimize import *
import random
import pandas as pd

import scipy

import math

from decimal import Decimal

from random import randint
lambertw = scipy.special.lambertw

### Contents

** 0.** Calculating $E_0$ and $\Delta$ from binding free energies

**I.** ssDNA - ssDNA interaction calculations

- (a) Free Energy Estimate

- (b) Calculating $(N)_{\text{max}}$, $(NV)_{\text{max}}$, and Type I inequality

**II.** TF-DNA Free interaction calculations

- (a) Free Energy Estimate

- (b) Calculating $(N)_{\text{max}}$, $(NV)_{\text{max}}$, and Type I inequality

**III.** Protein-protein interaction calculations

- (a) Free Energy Estimate

- (b) Calculating $(N)_{\text{max}}$, $(NV)_{\text{max}}$, and Type I inequality

### 0. Calculating $E_0$ and $\Delta$ from binding free energies

In the subsequent code, we determine the energy parameters $E_0$ and $\Delta$ from estimates of biomolecular binding free energies as follows. 


For a binding/association reaction, we define changes in Gibbs free energy as $\delta G$ and changes in entropy as $\delta S$. Thermodynamically, Gibbs free energy is distinct from Helmholtz free energy the latter of which is defined in terms of the former as $F = G + PV$, where $p$ and $V$ are system pressure and volume respectively. However, for systems at constant pressure and volume (as is presumed for our biomolecular dimerization systems), $\delta p$ and $\delta V$ are both zero, and changes in Helmholtz free energy equal changes in Gibbs free energy, $\delta G = \delta F$. Therefore we take $\delta G = \delta E - T \delta S$, where $\delta E$ is the change in energy upon binding. Solving for $\delta E$, we then obtain 

$$\delta E = \delta G + T\delta S.$$

In particular, for the model, the change in energy for a correct/specific interaction is $(\delta E)_{\text{correct}} = -(E_0+\Delta)$ and the change in energy for an incorrect/nonspecific interaction is $(\delta E)_{\text{incorrect}} = - E_0$. Therefore, we have 

\begin{eqnarray}
-(E_0+\Delta) & = (\delta G)_{\text{correct}} + T(\delta S)_{\text{incorrect}},\\
-E_0 & = (\delta G)_{\text{incorrect}} + T(\delta S)_{\text{incorrect}}.
\end{eqnarray}

In our model the only contribution to the change in entropy upon binding of two monomers is the change in translational entropy, and this change in translational entropy is independent of whether the binding is correct or incorrect. Therefore, we take 

$$ (\delta S)_{\text{correct}}  = (\delta S)_{\text{incorrect}} \equiv (\delta S)_{\text{trans}} $$

and the equations become 

\begin{eqnarray}
-(E_0+\Delta) & = (\delta G)_{\text{correct}} + T(\delta S)_{\text{trans}}, \\
-E_0 & = (\delta G)_{\text{incorrect}} + T(\delta S)_{\text{trans}}.
\end{eqnarray}

From these two equations, we find that $\Delta$ can be computed from the equation 

$$ \Delta = (\delta G)_{\text{incorrect}}- (\delta G)_{\text{correct}},$$

and $E_0$ is computed from 

$$ E_0 =  -(\delta G)_{\text{incorrect}} - T(\delta S)_{\text{trans}}.$$

The binding free energy $(\delta G)_{\text{incorrect}}$ is what we obtain from the biophysical data. We obtain $T(\delta S)_{\text{trans}}$ from the corresponding ideal gas expression. Specifically, the Sackur-Tetrode equation states that the entropy of an ideal solution (written in kcal/mol) is

\begin{equation}
T S/N_A = R T \ln \left[ e^{5/2}\frac{V_{\text{st}}}{N_A\lambda^3} \right]
\end{equation}

where $\lambda = h/\sqrt{2\pi m_0 k_BT}$ is the thermal de Broglie wavelength of the particle of interest, $N_A = 6.022 \times 10^{23}$ Avogadro's number,  $V_{\text{st}} = $ 1 L (standard volume of ideal solutions), and $R = 1.987\times10^{-3}$ kcal/K$^{-1}$mol$^{-1}$. We define the entropy in terms of $V_{\text{st}}$ and $N_A$ in order to match the conditions under which the binding free energy is defined; $\delta G$ is defined in units of kcal/mol i.e., in units of kcal per Avogadro's number of particles in a 1 L volume. 

As a general example, consider two motile monomers $A$ and $B$ which form the dimer $AB$. If the corresponding thermal de Broglie wavelengths of $A$, $B$, and $AB$ are $\lambda_{\alpha}$, $\lambda_{\beta}$ and $\lambda_{\alpha\beta}$, then the change in entropy upon binding is  

\begin{eqnarray}
T \delta S_{\text{trans}}/ N_{A} = T (S_{\alpha\beta}- S_{\alpha} - S_{\beta})/N_A = -RT \ln \left[e^{5/2}\frac{V_{\text{st}}}{\lambda_{\alpha}^3}\frac{\lambda_{\alpha\beta}^3}{\lambda_{\beta}^3}  \right]
\end{eqnarray}

We later apply this formula to calculate the binding energy parameters for our three systems of interest. 

It is known that for real protein systems, the ideal-solution approximation for entropy leads to an overestimate in the change in translational entropy upon binding since real protein monomers rarely explore their entire surrounding volume [1-2]. Thus, we can interpret our chosen entropy as providing an upper limit on the translational entropy contribution to the free energy and thus a lower limit on the obtained binding energy parameters in the model. 



- [1] Murphy, K. P., Xie, D., Thompson, K. S., Amzel, L. M., & Freire, E. (1994). Entropy in biological binding processes: estimation of translational entropy loss. *Proteins: Structure, Function, and Bioinformatics*, 18(1), 63-67.

- [2] Amzel, L. M. (1997). Loss of translational entropy in binding, folding, and catalysis. *Proteins: Structure, Function, and Bioinformatics*, 28(2), 144-149.


___________
___________

### I. ssDNA-ssDNA interaction calculations

#### (a). Free Energy Estimate

We develop code to calculate the free energy of binding for ssDNA+ssDNA(complement) to dsDNA given the sequence of one of the strands. 

This code is based on the results in 

* SantaLucia, J. (1998). A unified view of polymer, dumbbell, and oligonucleotide DNA nearest-neighbor thermodynamics. *Proceedings of the National Academy of Sciences, 95(4), 1460-1465.

The main equation we will be using is 

$$ \Delta G^{\circ}(\text{total}) = \sum_{i}n_i \Delta G^{\circ}(i) + \Delta G^{\circ}(\text{with term./ A-T})+\Delta G^{\circ}(\text{with term./ G-C})+ \Delta G^{\circ}(\text{sym.})$$

where $n_i$ is the number of nearest-neighbor base-pair interactions of type $i$, and $\Delta G^{\circ}(i)$ is the corresponding free energy; $\Delta G^{\circ}(\text{with term./ A-T})$ and $\Delta G^{\circ}(\text{with term./ G-C})$ are the additional free energies added for double strands ending in A-T and G-C respectively; $\Delta G^{\circ}(\text{sym.})$ is added to the result for self-complementary sequences. This equation applies for a 1M Na$^+$ solution.

Here is the table of free energies for these interactions at $37^{\circ}$ C. 

| Sequence | $\Delta G(i)$, kcal/mol|
| :---: | :---:  |
| AA/TT | -1.00  |
| AT/TA | -0.88  |
| TA/AT | -0.58  |
| CA/GT | -1.45  |
| GT/CA | -1.44  |
| CT/GA | -1.28  |
| GA/CT | -1.30  |
| CG/GC | -2.17  |
| GC/CG | -2.24  |
| GG/CC | -1.84  |
|Terminal G-C| 0.98  | 
|Terminal A-T| 1.03  |

**NOTE:** For a system with an arbitrary salt concentration we use the formula 

$$ \Delta G^{\circ}_{37}(\text{oligomer}, [\text{Na}^{+}]) = \Delta G^{\circ}_{37}(\text{oligomer}, 1 \text{ M NaCl}) - 0.114 \times N \times \ln [ \text{Na}^{+}], $$ 

where $N$ is the number of total phosphates in the duplex divided by 2.

##### Preliminary definitions

In [2]:
# Takes a 5' to 3' sequence and outputs 
# the complementary sequence in the 5' to 3' direction

def reverse(s): 
    st = "" 
    for i in s: 
        st = i + st
    return st

def comp_seq(string):
    
    empt_str = ''
    for k in string:
        if k == 'A':
            
            empt_str = empt_str +'T'
        elif k == 'T':
            
            empt_str = empt_str + 'A'
        
        elif k == 'G':
            
            empt_str = empt_str +'C'
            
        elif k == 'C':
            
            empt_str = empt_str +'G'
            
     
    return(reverse(empt_str))


In [3]:
## dictionary of energies for various sequences
## From Last column of Table 1 of referenced paper

en_dict = {
    "AA": -1.00,
    "TT": -1.00,
    "AT": -0.88,
    "TA": -0.58,
    "AG": -1.28,
    "AC": -1.44,
    "TG": -1.45,
    "TC": -1.30,
    "CA": -1.45,
    "GT": -1.44,
    "CT": -1.28,
    "GA": -1.30,
    "CG": -2.17,
    "GC": -2.24,
    "GG": -1.84,
    "CC": -1.84
}

In [4]:
# testing dictionary
en_dict['GC']

-2.24

##### ΔG of binding in 1M salt solution

In [5]:
## computes free energy in kcal/mol of DNA duplex in a 1M salt solution
## From Eq. 1 of referenced paper


def delG(string):
    
    Gtot = 0
    
    init = string[0]
    fin = string[len(string)-1]
    
    for k in range(len(string)-1):

        min_str = string[k:k+2]
        Gtot = Gtot + en_dict[min_str]
    
    # A-T terminal pair additional energy
    if fin == 'A' or fin == 'T':
        
        Gtot = Gtot + 1.03
        
    # G-C terminal pair additional energy
    elif fin == 'G' or fin == 'C':
        
        Gtot = Gtot + 0.98
        
    # A-T initial pair additional energy
    if init == 'A' or init == 'T':
        
        Gtot = Gtot + 1.03
        
    
    # G-C initial pair additional energy
    elif init == 'G' or init == 'C':
        
        Gtot = Gtot + 0.98
        
    ## additional free energy for self-complementarity
    if string == reverse(string):
        
        Gtot = Gtot + 0.43
        
    return Gtot

In [6]:
## testing calculation on sequence and corresponding value found in paper 
delG('CGTTGA')

-5.35

In [7]:
## testing that delta G gives same energy for sequence and complement

test_seq = 'CGCTAGCATCAGTCAGTCAGT'
delG(test_seq)- delG(comp_seq(test_seq))

-7.105427357601002e-15

In [8]:
## randomly generate 100000 20-base-pair sequences for ssDNA

bp_list = 'CGTA'

leng = 100000
seq_vec = ['']*leng
for k in range(leng):
    
    for j in range(20):
        u = randint(0,3)
        
        seq_vec[k] = seq_vec[k]+bp_list[u]

In [9]:
# test; compare with result in http://biophysics.idtdna.com/
delG('CGCTAGCATCAGTCAGTCAGT')

-26.520000000000007

In [10]:
## computing delta G for our list of sequences 
## in 1M salt solution

delG_list = np.ones(leng)

for k in range(leng):
    
    delG_list[k] = delG(seq_vec[k])

In [11]:
## computing mean and variance of ΔGs in 1M salt solution

print('Mean ΔGs (1M salt):',np.mean(delG_list))
print('Standard Deviation of ΔGs (1M salt):', np.std(delG_list))

Mean ΔGs (1M salt): -24.6898714
Standard Deviation of ΔGs (1M salt): 2.56563680856


##### ΔG of binding in arbitary salt solution

In [12]:
## calculation of ΔG with an incoporation of salt concentration 
## From Eq. 7 of referenced paper

def delG_salt(string, salt_concent):
    
    # salt_concentration is in units of molarity
    # e.g., 1M -> salt_concent  = 1
    
    # number of phosphates in the duplex divided by 2
    # assumes each nucleotide has a phosphate
    N = len(string)
    
    return delG(string) -0.114*N*np.log(salt_concent)

In [13]:
# test; compare with result in http://biophysics.idtdna.com/
delG_salt('ACTGACTGACTGATGCTAGCG', 0.05)

-19.348216937111744

In [14]:
## computing delta G for our list of sequences 
## in 1M salt solution

delG_list_salt = np.ones(leng)

for k in range(leng):
    
    delG_list_salt[k] = delG_salt(seq_vec[k], 0.05)

In [15]:
## computing mean and variance of ΔGs in 50 mM salt solution

print('Mean ΔGs (50 mM salt):',np.mean(delG_list_salt))
print('Standard Deviation of ΔGs (50 mM salt):', np.std(delG_list_salt))

Mean ΔGs (50 mM salt): -17.8596018163
Standard Deviation of ΔGs (50 mM salt): 2.56563680856


_____


#### (b). Calculating $(N)_{\text{max}}$, $(NV)_{\text{max}}$, and Type I inequality

We calculate $(N)_{\text{max}}$, $(NV)_{\text{max}}$, and the right-hand-side of the "Type I inequality" for the TF-DNA system.

** Relevant Inequalities **

\begin{eqnarray*}
(NV)_{\text{max}} = \sqrt{2} \lambda_{0}^3 e^{\beta(\Delta+E_0)}, \qquad \text{[$T= T_{\text{I}}$]}
\end{eqnarray*}
where $\lambda_{0} = h/\sqrt{2\pi m_0 k_BT}$

\begin{eqnarray*}
(N)_{\text{max}} = e^{\beta \Delta}/2, \qquad \text{[$T= T_{\text{II}}$]}
\end{eqnarray*}


\begin{eqnarray*}
2N < \exp\left[\frac{3\Delta}{2E_0} W_{0}\left(\frac{E_0}{3E_V}\right)\right], \qquad \text{[Type I inequality, $T_{\text{I}}< T_{\text{II}}$]}
\end{eqnarray*}

where $E_{ V} = h^2/2\pi m_0 V^{2/3}$.

**Change in Translational Entropy upon Binding**

\begin{eqnarray}
T\left(S_{\text{dsDNA}} - 2\,S_{\text{ssDNA}}\right)/N_A & = R T \ln \left[e^{5/2}\frac{V_{\text{st}}}{N_A(\lambda_0/\sqrt{2})^3} \right] - 2 R T \ln \left[e^{5/2}\frac{V_{\text{st}}}{N_A\lambda_0^3} \right]\\
& = -R T \ln \left[e^{5/2}\frac{V_{\text{st}}}{2\sqrt{2}\,N_A\lambda_0^3} \right] \\
& = -13.6 \text{ kcal/mol}.
\end{eqnarray}

In [16]:
# Computing translational entropy contribution to free energy

# We use SI units throughout, except for energies which are defined in kcal/mol

Tphys = 310.15 # physiological temperature
RTphys = 0.001987*Tphys # Gas constant times temperature
V_stan = 1e-3 # 1 liter in meters cubed
kB = 1.38*10**(-23) # Boltzmann's constant
mass = 1.0793504*10**(-23) # 6500 Daltons mass of 20-base strand of ssDNA
Na = 6.022*10**23 # Avogadro's number
h = 6.626*10**(-34) # Planck's constant
lam_0 = h/np.sqrt(2*np.pi*mass*kB*Tphys) # thermal deBroglie wavelength

-RTphys*np.log(np.exp(5/2)*V_stan/(2*np.sqrt(2)*Na*lam_0**3))

-13.601045435350864

** Energy Parameter Estimation **

From the specific free energy of binding we find: 

$$ \delta E = \delta G + T \delta S = -17.9 \text{ kcal/mol} - 13.6 \text{ kcal/mol} = -31.5  \text{ kcal/mol} = -(E_0+\Delta) =  -\Delta$$

where we took $E_0= 0$ in the final equality. 

In [17]:
## Calculation (N)max, (NV)max, and Type I inequality

# System paramaters for DNA system
kB = 1.38*10**(-23) # boltzmann's constant
mass = 1.0793504*10**(-23) # 6500 Daltons mass of 20-base strand of ssDNA
Na = 6.022*10**23 # Avogadro's number
h = 6.626*10**(-34) # Planck's constant

E0 = 1e-5 # E0=0, 
Del = 31.5
Tphys = 310.15 # physiological temperature
RTphys = 0.001987*Tphys # Gas constant times temperature
Vol = 1e-18 # we assume a volume of (1 micrometer)^3

lam_0_phys = h/np.sqrt(2*np.pi*mass*kB*Tphys) # thermal deBroglie wavelength at phys temperature

Ev = h**2/(2*np.pi*mass*Vol**(2/3))*(2.388e-4)*Na # Ev written in kcal/mol

Nmax = np.exp(Del/RTphys)/2
NVmax = np.sqrt(2)*lam_0_phys**3*np.exp((E0+Del)/RTphys)/(1e-18)
# Type1_ineq = np.exp((3*Del)/(2*E0)*lambertw(E0/(3*Ev))).real  #overflow error

print("(N)max =", '%.2E' % Decimal(Nmax))
print("(NV)max=", '%.2E' % Decimal(NVmax))
#print("exp((3Δ/2E0) W0(E0/3EV))=", '%.2E' % Decimal(Type1_ineq))#overflow error

(N)max = 7.90E+21
(NV)max= 4.16E+04


In [18]:
# Argument of Type1_ineq exponential
'%.2E' % Decimal(Del/(h**2/(2*np.pi*mass*Vol**(2/3))*(2.388e-4)*Na))

'3.38E+13'

_____

----

### I. TF-DNA interaction calculations

#### (a). Free Energy Estimate

We estimate the specific and non-specific free energy of binding between TFs and DNA for some representative interactions

Data is taken from Table 1 in 

* Jen‐Jacobson, L. (1997). Protein—DNA recognition complexes: Conservation of structure and binding energy in the transition state. *Biopolymers: Original Research on Biomolecules*, 44(2), 153-180.

In [19]:
# 12 TFs and their specific and non-specific binding energies to DNA
'''
Transcribed from Table 1 in:

Jen‐Jacobson, Linda. 
"Protein—DNA recognition complexes: Conservation of structure 
and binding energy in the transition state." 
Biopolymers: Original Research on Biomolecules 44.2 (1997): 153-180.
'''
protein_DNA_dfs = pd.read_excel('protein_DNA_affinity.xlsx', sheet_name=None)

In [20]:
# List of column names 
list(protein_DNA_dfs)

['Protein ',
 'Specific Binding Constants',
 'Non-Specific Binding Constants ',
 'Temperature',
 'Specific Free energy (presuming room temperature)',
 'Specific Free energy (temperature corrected value)',
 'Nonspecific Free Energy (presuming room temperature)',
 'Nonspecific Free Energy (temperature corrected value)']

In [21]:
# Sample Data 
protein_DNA_dfs.head()

,Protein,Specific Binding Constants,Non-Specific Binding Constants,Temperature,Specific Free energy (presuming room temperature),Specific Free energy (temperature corrected value),Nonspecific Free Energy (presuming room temperature),Nonspecific Free Energy (temperature corrected value)
0,λ Cro,500000000000,680000.0,273.15,15.960690,14.622004,7.957185,7.289784
1,-,120000000000,NaN,273.15,15.115124,13.847358,NaN,NaN
2,-,8300000000,NaN,273.15,13.532416,12.397399,NaN,NaN
3,λ CcI repressor,1300000000,12000.0,295.15,12.433986,12.308558,5.565152,5.509014
4,-,8300000000,NaN,293.15,13.532416,13.305134,NaN,NaN


In [22]:
# Data table; Mean of the specific free energy in kcal/mol
protein_DNA_dfs['Specific Free energy (temperature corrected value)'].mean()

12.915483849237445

In [23]:
# Data table; Mean of the non-specific free energy in kcal/mol
protein_DNA_dfs['Nonspecific Free Energy (temperature corrected value)'].mean()

6.381468009815689

_____


#### (b). Calculating $(N)_{\text{max}}$, $(NV)_{\text{max}}$, and Type I inequality

We calculate $(N)_{\text{max}}$, $(NV)_{\text{max}}$, and the right-hand-side of the "Type I inequality" for the TF-DNA system. In contrast to the ssDNA-ssDNA and protein-protein interaction calculations, we use the results for a gendered system. 

** Relevant Equalities/Inequalities **

\begin{eqnarray*}
(NV)_{\text{max}} = \lambda_{\mu}^3 \,e^{\beta(\Delta+E_0)}\qquad \text{[$T= T_{\text{I}}'$]},
\end{eqnarray*}
where $\lambda_{\mu} = h/\sqrt{2\pi \mu k_BT}$

\begin{eqnarray*}
(N)_{\text{max}} = e^{\beta \Delta}\qquad \text{[$T= T_{\text{II}}'$]},
\end{eqnarray*}


\begin{eqnarray*}
N < \exp\left[\frac{3\Delta}{2E_0} W_{0}\left(\frac{2E_0}{3E_{\mu, V}}\right)\right],  \quad \text{[Type I inequality, $T_{\text{I}}< T_{\text{II}}'$]}
\end{eqnarray*}

where $E_{\mu, V} = h^2/2\pi \mu V^{2/3}$.

**Change in Translational Entropy upon Binding**

\begin{equation}
 T_{\text{st}}\left(S_{\text{TF-DNA}} -\,S_{\text{TF}}-\,S_{\text{DNA}}\right)/N_A =  -T_{\text{st}} S_{\text{TF}}/N_A =  - RT_{\text{st}} \ln \left[e^{5/2}\frac{V_{\text{st}}}{N_A\lambda_{\text{TF}}^3} \right] = - 16.4 \text{ kcal/mol},
\end{equation}

where we took $S_{\text{TF-DNA}} = S_{\text{DNA}}$ since the DNA (binding site) is fixed compared to the motile TF. 

In [24]:
# Computing translational entropy contribution to free energy

# We use SI units throughout, except for energies which are defined in kcal/mol

Tphys = 310.15 # physiological temperature
RTphys = 0.001987*Tphys # Gas constant times temperature
V_stan = 1e-3 # 1 liter in meters cubed
kB = 1.38*10**(-23) # Boltzmann's constant
mass = 1.062745*10**(-22) # 64 kDa; mass of protein
Na = 6.022*10**23 # Avogadro's number
h = 6.626*10**(-34) # Planck's constant
lam_0 = h/np.sqrt(2*np.pi*mass*kB*Tphys) # thermal deBroglie wavelength

-RTphys*np.log(np.exp(5/2)*V_stan/(Na*lam_0**3))

-16.355974454934454

** Parameter Estimation **

From the specific free energy of binding we find:

$$ \delta E = \delta G + T \delta S = -12.9 \text{ kcal/mol} - 16.4 \text{ kcal/mol} = -29.3  \text{ kcal/mol} = -(E_0+\Delta),$$

and from the non-specific free energy of binding we have $$-\Delta = -6.4\text{ kcal/mol}.$$

In [25]:
## Calculation (N)max, (NV)max, and Type I inequality

# System paramaters for DNA system
kB = 1.38*10**(-23) # Boltzmann's constant
mass = 1.062745*10**(-22) # 64 kDa; mass of protein
h = 6.626*10**(-34) # Planck's constant
Na = 6.022*10**23 # Avogadro's number

E0 = 29.3-6.4
Del = 6.4
Tphys = 310.15 # physiological temperature
RTphys = 0.001987*Tphys # Gas constant times temperature
Vol = 1e-18 # we assume a volume of (1 micrometer)^3

lam_0_phys = h/np.sqrt(2*np.pi*mass*kB*Tphys) # thermal deBroglie wavelength at phys temperature

Ev = h**2/(2*np.pi*mass*Vol**(2/3))*(2.388e-4)*Na # Ev written in kcal/mol

Nmax = np.exp(Del/RTphys)
NVmax = lam_0_phys**3*np.exp((E0+Del)/RTphys)/(1e-18)
Type1_ineq = np.exp((3*Del)/(2*E0)*lambertw(2*E0/(3*Ev))).real 

print("(N)max =", '%.2E' % Decimal(Nmax))
print("(NV)max=", '%.2E' % Decimal(NVmax))
print("exp((3Δ/2E0) W0(2E0/3EV))=", '%.2E' % Decimal(Type1_ineq))

(N)max = 3.24E+04
(NV)max= 2.68E+01
exp((3Δ/2E0) W0(2E0/3EV))= 2.19E+05


-----

------

### III. Protein-protein interaction calculations

#### (a). Free Energy Estimate


We estimate the specific free energy of binding for protein-protein intereactions for some representative interactions

Data is taken from the "Affinity Benchmark 2.0 Values" here https://bmm.crick.ac.uk/~bmmadmin/Affinity/

* Kastritis, P. L., Moal, I. H., Hwang, H., Weng, Z., Bates, P. A., Bonvin, A. M., & Janin, J. (2011). A structure‐based benchmark for protein–protein binding affinity. *Protein Science*, 20(3), 482-491.

Difference between non-specific an specific binding (i.e., $(\delta G)_{\text{non-spec.}}- (\delta G)_{\text{spec.}}$) is found to be $12.5\,RT$ (last paragraph of pg. 5) or $7.7$ kcal/mol for $T = 310.15$ K. 

* Zhang, J., Maslov, S., & Shakhnovich, E. I. (2008). Constraints imposed by non‐functional protein–protein interactions on gene expression and proteome size. *Molecular systems biology*, 4(1), 210.

In [26]:
# 12 TFs and their specific and non-specific binding energies to DNA
'''
Data from:

Protein-Protein Affinity Database: https://bmm.crick.ac.uk/~bmmadmin/Affinity/

associated with paper 

Kastritis, Panagiotis L., et al. "A structure‐based benchmark 
for protein–protein binding affinity." Protein Science 20.3 (2011): 482-491.
'''
protein_protein_dfs = pd.read_excel('protein_protein_affinity.xlsx', sheet_name=None)

In [27]:
# List of column names 
list(protein_protein_dfs)

['Complex PDB',
 'Type',
 'Unbound PDB',
 'Protein A',
 'Unbound PDB.1',
 'Protein B',
 'Pubmed',
 'Kd (M)',
 'dG',
 'I-RMSD',
 'DASA ',
 'Method',
 'Final Comments',
 'Temperature (C)',
 'pH',
 'Buffer/Ionic Strength',
 'Cofactors (bound)',
 'Cofactors (unbound)',
 'Cofactors (binding)',
 'Extra Data; NB) kon in mol^-1 s^-1, koff in s^-1, dH in kcal mol^-1 dS and dCp in cal mol^-1 K^-1',
 'Corroborating Data',
 'Exact finding in: ']

In [28]:
# Sample Data 
protein_protein_dfs.head()

,Complex PDB,Type,Unbound PDB,Protein A,Unbound PDB.1,Protein B,Pubmed,Kd (M),dG,I-RMSD,...,Final Comments,Temperature (C),pH,Buffer/Ionic Strength,Cofactors (bound),Cofactors (unbound),Cofactors (binding),"Extra Data; NB) kon in mol^-1 s^-1, koff in s^-1, dH in kcal mol^-1 dS and dCp in cal mol^-1 K^-1",Corroborating Data,Exact finding in:
0,1A2K_C:AB,OG,1QG4_A,Ran GTPase-GDP,1OUN_AB,Nuclear transport factor 2,10681579.0,1.5e-07,-9.31,1.11,...,NaN,25,7.5,1,GDP/Mg,GDP/Mg,GDP/Mg,Microtitre plates: Kd = 2.4e-7. Fluorescence:...,NaN,Fig. 1
1,1ACB_E:I,EI,4CHA_ABC,Chymotrypsin,1EGL_A,Eglin C,3071573.0,2e-10,-13.05,1.08,...,calculated from Ka,21,8,1,NaN,NaN,NaN,"DG(STP) = -13.0, dH(STP) = +2.0, dS(STP) = +51...","PMID 1741752: Kd = 4e-13, (50mM Hepes, 100mM ...","Table 1, calculated from Ka; PMID 1741752: p.31"
2,1AHW_AB:C,A,1FGN_LH,Fab 5g9,1TFH_A,Tissue factor,9480775.0,3.4e-09,-11.55,0.69,...,NaN,ambient,not stated,1,NaN,NaN,NaN,NaN,NaN,Table 4
3,1AK4_A:D,OX,2CPL_A,Cyclophilin,4J93_A,HIV capsid,9223641.0,1.6e-05,-6.43,1.33,...,NaN,20,6.5,1,NaN,NaN,NaN,SPR (same conditions): Kd = 1.8e-5. Equillibr...,"PMID 19767750 (ITC): Kd = 5e-5, dCp = -285 dH...","Fig. 4; PMID 19767750: Fig.2, PMID 8980234: p...."
4,1AKJ_AB:DE,OX,2CLR_DE,MHC class 1 HLA-A2,1CD8_AB,T-cell CD8 coreceptor,10072074.0,0.000126,-5.32,1.14,...,NaN,25,7.4,1,NaN,NaN,NaN,"Kon ? 140000, koff ? 18 s?1",NaN,Table 1


In [29]:
# Data table; Mean of the binding free energy in kcal/mol
protein_protein_dfs['dG'].mean()

-10.877608695652171

_____


#### (b). Calculating $(N)_{\text{max}}$, $(NV)_{\text{max}}$, and Type I inequality

We calculate $(N)_{\text{max}}$, $(NV)_{\text{max}}$, and the right-hand-side of the "Type I inequality" for protein-protein systems

** Relevant Inequalities **


\begin{eqnarray*}
(NV)_{\text{max}} = \sqrt{2} \lambda_{0}^3 e^{\beta(\Delta+E_0)}, \qquad \text{[$T= T_{\text{I}}$]}
\end{eqnarray*}
where $\lambda_{0} = h/\sqrt{2\pi m_0 k_BT}$

\begin{eqnarray*}
(N)_{\text{max}} = e^{\beta \Delta}/2, \qquad \text{[$T= T_{\text{II}}$]}
\end{eqnarray*}

\begin{eqnarray*}
2N < \exp\left[\frac{3\Delta}{2E_0} W_{0}\left(\frac{E_0}{3E_V}\right)\right], \qquad \text{[Type I inequality, $T_{\text{I}}< T_{\text{II}}$]}
\end{eqnarray*}

where $E_{ V} = h^2/2\pi m_0 V^{2/3}$.

**Change in Translational Entropy upon Binding**


\begin{equation}
 T_{\text{st}}\left(S_{\text{protein-protein}} - 2 S_{\text{protein}}\right)/N_A = - R T_{\text{st}} \ln \left[e^{5/2}\frac{V_{\text{st}}}{2\sqrt{2}\,N_A\lambda_{0}^3} \right] = - 15.7 \text{ kcal/mol}.
\end{equation}

In [30]:
# Computing translational entropy contribution to free energy

# We use SI units throughout, except for energies which are defined in kcal/mol

Tphys = 310.15 # physiological temperature
RTphys = 0.001987*Tphys # Gas constant times temperature
V_stan = 1e-3 # 1 liter in meters cubed
kB = 1.38*10**(-23) # Boltzmann's constant
mass = 1.062745*10**(-22) # 64 kDa; mass of protein
Na = 6.022*10**23 # Avogadro's number
h = 6.626*10**(-34) # Planck's constant
lam_0 = h/np.sqrt(2*np.pi*mass*kB*Tphys) # thermal deBroglie wavelength

-RTphys*np.log(np.exp(5/2)*V_stan/(2*np.sqrt(2)*Na*lam_0**3))

-15.715227762944441

** Parameter Estimation **

From the free energy of binding (presumed to be specific) we find:

$$ \delta E = \delta G + T \delta S = -10.9 \text{ kcal/mol} - 15.7 \text{ kcal/mol} = -26.6  \text{ kcal/mol} = -(E_0+\Delta)$$

and from Zhang *et.al.* refernce we have $$-\Delta = -7.7\text{ kcal/mol}$$

In [31]:
## Calculation (N)max, (NV)max, and Type I inequality

# System paramaters for DNA system
kB = 1.38*10**(-23) # Boltzmann's constant
mass = 1.062745*10**(-22) # 64 kDa; mass of protein
h = 6.626*10**(-34) # Planck's constant
Na = 6.022*10**23 # Avogadro's number

E0 = 26.6-7.7
Del = 7.7
Tphys = 310.15 # physiological temperature
RTphys = 0.001987*Tphys # Gas constant times temperature
Vol = 1e-18 # we assume a volume of (1 micrometer)^3

lam_0_phys = h/np.sqrt(2*np.pi*mass*kB*Tphys) # thermal deBroglie wavelength at phys temperature

Ev = h**2/(2*np.pi*mass*Vol**(2/3))*(2.388e-4)*Na # Ev written in kcal/mol

Nmax = np.exp(Del/RTphys)/2
NVmax = np.sqrt(2)*lam_0_phys**3*np.exp((E0+Del)/RTphys)/(1e-18)
Type1_ineq = np.exp((3*Del)/(2*E0)*lambertw(E0/(3*Ev))).real 


print("(N)max =", '%.2E' % Decimal(Nmax))
print("(NV)max=", '%.2E' % Decimal(NVmax))
print("exp((3Δ/2E0) W0(E0/3EV))=", '%.2E' % Decimal(Type1_ineq))

(N)max = 1.33E+05
(NV)max= 4.74E-01
exp((3Δ/2E0) W0(E0/3EV))= 3.62E+07
